### Assignment 8

In [1]:
import json

from kafka import KafkaConsumer

### Configuration Parameters 

> **TODO:** Change the configuration prameters to the appropriate values for your setup.

In [2]:
config = dict(
    bootstrap_servers=['kafka.kafka.svc.cluster.local:9092'],
    first_name='Curry',
    last_name='Adam'
)

config['client_id'] = '{}{}'.format(
    config['last_name'], 
    config['first_name']
)
config['topic_prefix'] = '{}{}'.format(
    config['last_name'], 
    config['first_name']
)

config

{'bootstrap_servers': ['kafka.kafka.svc.cluster.local:9092'],
 'first_name': 'Curry',
 'last_name': 'Adam',
 'client_id': 'AdamCurry',
 'topic_prefix': 'AdamCurry'}

In [3]:
config = dict(
    bootstrap_servers=['kafka.kafka.svc.cluster.local:9092'],
    first_name='Adam',
    last_name='Curry'
)

config['client_id'] = '{}{}'.format(
    config['last_name'], 
    config['first_name']
)
config['topic_prefix'] = '{}{}'.format(
    config['last_name'], 
    config['first_name']
)

config

{'bootstrap_servers': ['kafka.kafka.svc.cluster.local:9092'],
 'first_name': 'Adam',
 'last_name': 'Curry',
 'client_id': 'CurryAdam',
 'topic_prefix': 'CurryAdam'}

Create a consumer without subscribing to any particular topic

In [4]:
general_consumer = KafkaConsumer(
    bootstrap_servers=config['bootstrap_servers']
)

List all topics you are currently allowed to view

In [5]:
general_consumer.topics()

{'AdamCurry-accelerations',
 'AdamCurry-locations',
 'AtmadjaRio-accelerations',
 'AtmadjaRio-joined',
 'AtmadjaRio-locations',
 'AtmadjaRio-simple',
 'AtmadjaRio-windowed',
 'BranchJolene-accelerations',
 'BranchJolene-locations',
 'BriggsChris-acceleration',
 'BriggsChris-accelerations',
 'BriggsChris-locations',
 'ByardMegan-accelerations',
 'ByardMegan-locations',
 'CorpuzCaleb-accelarations',
 'CorpuzCaleb-accelerations',
 'CorpuzCaleb-locations',
 'CurryAdam-accelerations',
 'CurryAdam-locations',
 'DandaRama-accelerations',
 'DandaRama-locations',
 'DoeJohn-accelerations',
 'DoeJohn-locations',
 'DoeJohn-simple',
 'DoeJohn-windowed',
 'DoeagerJohn-accelerations',
 'DoeagerJohn-locations',
 'EagerBrian-accelerations',
 'EagerBrian-locations',
 'EdmundsEvan-accelerations',
 'EdmundsEvan-locations',
 'FoxAndrea-accelerations',
 'FoxAndrea-locations',
 'HarveyAnna-locations',
 'HerbstreitSara-accelerations',
 'HerbstreitSara-locations',
 'IsmailJason-accelerations',
 'IsmailJason-lo

Close the consumer, waiting indefinitely for any needed cleanup.

In [6]:
general_consumer.close()

In [9]:
def create_kafka_consumer(topics, config=config):
    bootstrap_servers = config['bootstrap_servers']
    client_id = config['client_id']
    topic_prefix = config['topic_prefix']    
    topic_list = ['{}-{}'.format(topic_prefix, topic) for topic in topics]
    
    return KafkaConsumer(
        *topic_list,
        client_id=client_id,
        bootstrap_servers=bootstrap_servers,
        auto_offset_reset='earliest',
        enable_auto_commit=False,
        value_deserializer=lambda x: json.loads(x)
    )

consumer = create_kafka_consumer(['locations', 'accelerations'])

Gets a list of this consumer's current subscriptions

In [10]:
consumer.subscription()

{'CurryAdam-accelerations', 'CurryAdam-locations'}

The following function prints messages from the current consumer subscriptions. It will continue until manually stopped. 

In [11]:
def print_messages(consumer=consumer):
    try:
        for message in consumer:
                msg_metadata = 'Message metadata: {}:{}:{}'.format(
                    message.topic, message.partition, message.offset
                )

                if message.key is not None:
                    msg_key = message.key.decode('utf-8')
                else:
                    msg_key = ''
                msg_value = json.dumps(message.value, indent=2)
                msg_value = '\n'.join(['  {}'.format(value) for value in msg_value.split('\n')])

                print('Message metadata:')
                print('  Topic: {}'.format(message.topic))
                print('  Partition: {}'.format(message.partition))
                print('  Offset: {}'.format(message.offset))
                print('Message Key: {}'.format(msg_key))
                print('Message Value:')
                print(msg_value)
                print()
    except KeyboardInterrupt:
        print("STOPPING MESSAGE CONSUMER")
        
print_messages()

Message metadata:
  Topic: CurryAdam-locations
  Partition: 0
  Offset: 0
Message Key: 72bbf5ad5b644663912e53893d362b0f
Message Value:
  {
    "key1": "value1",
    "key2": "value2"
  }

Message metadata:
  Topic: CurryAdam-locations
  Partition: 0
  Offset: 1
Message Key: cd28f5225eb24683b4798c1147dbbe4a
Message Value:
  {
    "key1": "value1",
    "key2": "value2"
  }

Message metadata:
  Topic: CurryAdam-locations
  Partition: 0
  Offset: 2
Message Key: 0034178df0a54dd0a82cfb4b3d301366
Message Value:
  "{\"id\":{\"0\":\"58682c5d48cad9d9e103431d773615bf\",\"1\":\"85c61911b7fe2ced1000c33c9e932706\"},\"ride_id\":{\"0\":\"c9a2b46c9aa515b632eddc45c4868482\",\"1\":\"6760ffa3f41908695d1405b776c3e8d5\"},\"uuid\":{\"0\":\"19b9aa10588646b3bf22c9b4865a7995\",\"1\":\"dad7eae44e784b549c8c5a3aa051a8c7\"},\"timestamp\":{\"0\":1503882,\"1\":1507320},\"offset\":{\"0\":1.5250608865,\"1\":1.0779125296},\"course\":{\"0\":299.619140625,\"1\":158.203125},\"latitude\":{\"0\":40.7628700254,\"1\":40.6776413

Close the consumer, waiting indefinitely for any needed cleanup.

In [12]:
consumer.close()